# [Langsmith](https://python.langchain.com/v0.1/docs/langsmith/walkthrough/) + [Langgraph](https://langchain-ai.github.io/langgraph/#example) + [langchain_neospace](https://github.com/neospace-ai/langchain-neospace/releases)

## Setup

### Installation

In [12]:
%pip install -U langgraph langsmith
%pip install langchain-openai
%pip install python-dotenv
%pip install --upgrade --quiet  langchain langchainhub
%pip install --upgrade --quiet  tiktoken pandas duckduckgo-search
%pip install langchain_community
%pip install https://github.com/neospace-ai/langchain-neospace/releases/download/v0.1.18/langchain_neospace-0.1.18-py3-none-any.whl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.2/339.2 kB 3.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Env

In [13]:
from dotenv import load_dotenv, find_dotenv
import os
dotenv = find_dotenv()
print(f"Found .env file at {dotenv}")
load_dotenv(dotenv_path=dotenv, override=True)

from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"


Found .env file at /home/guilherme-alves-carvalho/Documents/repos/dsc-langgraph/.env


### Helpers

In [14]:
class LLMLogger:
    def __init__(self, wrapped_class):
        self.wrapped_class = wrapped_class
        run_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        os.makedirs(f"logs/{run_date}", exist_ok=True)

        self.req_res_f = f"logs/{run_date}/exec.json"

        self.req_res = []

    def write_to_file(self):
        with open(self.req_res_f, "w+") as f:
            json.dump(self.req_res, f, indent=4, ensure_ascii=False)
        

    def __getattr__(self, attr):
        original_func = getattr(self.wrapped_class, attr)

        def wrapper(*args, **kwargs):

            result = original_func(*args, **kwargs)

            self.req_res.append({"request": kwargs, "response": result.model_dump()})
            return result

        return wrapper

## Workflow

### Build

In [15]:
import datetime
from typing import Annotated, Literal

from langchain_core.tracers.context import tracing_v2_enabled
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool



from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

import os
import json

from langchain import hub
from langchain_community.tools import DuckDuckGoSearchResults
from langsmith import Client
from langchain_neospace import ChatNeoSpace
import json

# https://smith.langchain.com/hub/wfh/langsmith-agent-prompt?organizationId=3e9f24f6-5a49-5652-a085-d776844de3bf
# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")

# Define the tools for the agent to use
tools = [
    DuckDuckGoSearchResults(
        name="duck_duck_go"
    ),  # General internet search using DuckDuckGo
]

tool_node = ToolNode(tools)

model = ChatNeoSpace(
    model="7b-mistral-balanced_loan_math_tools-loan_no_tools-rank16-scaling2-dropout01-lora_all",
    temperature=0,
)

model.client = LLMLogger(model.client)

model = model.bind_tools(tools)

class State(MessagesState):
    metadata: Annotated[dict, "The configuration for the workflow"]

# Define the function that determines whether to continue or not
def should_continue(state: State) -> Literal["tools", "agent", END]:
    messages = state['messages']
    last_message = messages[-1]

    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"

    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: State):
    messages = state['messages']
    response = model.bind(extra_body=state['metadata']).invoke(messages)
    # We return a list, because this will get added to the existing list

    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

### Run

In [16]:
langsmith_client = Client(api_url=os.getenv("MERCURY_ENDPOINT"))
with tracing_v2_enabled(client=langsmith_client):
    metadata = {
        "customer": {
            "id": "45646",
            "name": {
                "value": "Felipe Silva",
                "searchable": True
            },
            "gender": {
                "value": "male",
                "searchable": False
            }
        },
        "group": {
            "id": "12345",
            "name": {
                "value": "Whatsapp",
                "searchable": True
            }
        }
    }

    question = HumanMessage(content=str(input()))
    state = {"messages": [SystemMessage(content="Você é um agente especialista financeiro, mostre todos os seus cálculos realizados em sua resposta"), question], "metadata": metadata}
     

    while question.content != "q":
        print(f"Human => {state['messages'][-1].content}")
        
        state = app.invoke(
            state,
            config={"thread_id": 42, "workflow": workflow},
        )
        print(f"AI => {state['messages'][-1].content}")
        
        question = HumanMessage(content=str(input()))
        state["messages"] = state["messages"] + [question]

    model.client.write_to_file()


Human => Ola, tudo bem?
AI =>  Estou em excelente forma, {{name}}! Como posso ser útil para você hoje?
Human => Gostaria de realizar um empréstimo
AI =>  Compreendi, {{name}}. Você já determinou a quantia que deseja pedir como empréstimo e o prazo que precisa para o seu pagamento?
Human => Não sei, acho q no valor de 140 reais
AI =>  Desafortunadamente, a ferramenta de análise que você acaba de solicitar não é disponível. No entanto, estou aqui para ajudar se você precisar de algo mais.
Human => ok, tchau
AI =>   Desafortunadamente, a ferramenta de execução de empréstimo que você acaba de solicitar não está disponível. No entanto, estou aqui para ajudar se você necessitar de algo mais.
